In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import time
import datetime
import re
import urllib
from bs4 import BeautifulSoup
from random import randint
import pyLDAvis.gensim
from pprint import pprint

In [3]:
initial_date_str = '1 Jan 2018'
initial_date = datetime.datetime.strptime(initial_date_str, '%d %b %Y')
final_date_str = '13 Jun 2018'

final_date = datetime.datetime.strptime(final_date_str, '%d %b %Y')
current_date = initial_date

In [4]:
tweets_list = []
while current_date < final_date :        
    aux_list = open('Tweets/' + current_date.strftime("%Y-%m-%d") + '_trump.txt',encoding='UTF-8').read().split('\n')
    aux_list = [current_date.strftime("%Y-%m-%d") + ', ' + row for row in aux_list]
    tweets_list.extend(aux_list)
    current_date = current_date + datetime.timedelta(days=1)

In [5]:
tweets_list[6500]

'2018-04-15, JTStevison, https://twitter.com/JTStevison/status/985656472687259649, 1, 11, 4, Perfect. <img class="Emoji Emoji--forText" src="https://abs.twimg.com/emoji/v2/72x72/1f602.png" draggable="false" alt="😂" title="Face with tears of joy" aria-label="Emoji: Face with tears of joy"><img class="Emoji Emoji--forText" src="https://abs.twimg.com/emoji/v2/72x72/1f602.png" draggable="false" alt="😂" title="Face with tears of joy" aria-label="Emoji: Face with tears of joy"><a href="/hashtag/DonaldTrump?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b><strong>DonaldTrump</strong></b></a> <a href="/hashtag/Syria?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>Syria</b></a> <a href="/hashtag/SyrianStrikes?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>SyrianStrikes</b></a> <a href="/hashtag/Trump?src=hash" data-qu

In [6]:
def add_space_between_a_tags(html):
    pos_1 = html.find('<a')
    while pos_1 != -1:
        html = html[:pos_1] + ' ' + html[pos_1:]
        pos_1 = html.find('<a',pos_1+2)
        
    pos_1 = html.find('</a>')
    while pos_1 != -1:
        html = html[:pos_1+4] + ' ' + html[pos_1+4:]
        #if len(html) == html.find('</a>',pos_1 + 4)
        pos_1 = html.find('</a>',pos_1+4)
        
    return html

In [7]:
print(add_space_between_a_tags(tweets_list[9870]))

2018-06-09, reggaerapstarCP, https://twitter.com/reggaerapstarCP/status/1005393266530021379, 0, 1, 3, Remember this?This This  <a href="/hashtag/Original?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>Original</b></a>   <a href="/hashtag/hand?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>hand</b></a>   <a href="/hashtag/painting?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>painting</b></a>   <a href="/hashtag/handdesigned?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>handdesigned</b></a>   <a href="/hashtag/usa?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>usa</b></a>   <a href="/hashtag/presidenttrump?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link j

In [8]:
print(tweets_list[9870])

2018-06-09, reggaerapstarCP, https://twitter.com/reggaerapstarCP/status/1005393266530021379, 0, 1, 3, Remember this?This This <a href="/hashtag/Original?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>Original</b></a> <a href="/hashtag/hand?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>hand</b></a> <a href="/hashtag/painting?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>painting</b></a> <a href="/hashtag/handdesigned?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>handdesigned</b></a> <a href="/hashtag/usa?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>usa</b></a> <a href="/hashtag/presidenttrump?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir=

In [9]:
def get_hashtags(tweet):
    hashtags = []
    pos = tweet.find('#')
    while pos != -1:
        pos_aux = tweet.find(" ",pos)
        if pos_aux == -1:
            break
        hashtags.append(tweet[pos:pos_aux])
        tweet = tweet[:pos] + ' ' + tweet[pos_aux:]
        pos = tweet.find('#',pos + 1)
    return hashtags

In [10]:
print(get_hashtags("Body Language Analysis No.4303: Donald Trump, G7 Summit, Women's Empowerment, and Rage -  #Nonverbal &  #EmotionalIntelligence •   •  #DonaldTrump #AngelaMerkel #Anger #BehaviorAnalysis #BodyLanguage #BodyLanguageExpert #Analysis #EQ #G7 #G7Summit"))

['#Nonverbal', '#EmotionalIntelligence', '#DonaldTrump', '#AngelaMerkel', '#Anger', '#BehaviorAnalysis', '#BodyLanguage', '#BodyLanguageExpert', '#Analysis', '#EQ', '#G7']


In [11]:
def get_mentions(tweet):
    mentions = []
    pos = tweet.find('@')
    while pos != -1:
        pos_aux = tweet.find(" ",pos)
        if pos_aux == -1:
            break
        mentions.append(tweet[pos:pos_aux])
        tweet = tweet[:pos] + ' ' + tweet[pos_aux:]
        pos = tweet.find('@',pos + 1)
    return mentions

In [12]:
print(get_mentions("May God Bless you and Bless God for  #DonaldTrump @lindathomas102 @louisa_ip @maga_417 @MarcGriff89 @MarjorieLoy1@Maximus_Paulus @MC_Mad_Man @mgw81180 @MrSteveRangel @MyMountainTop @Namashary @netteKnows @NotUnderdog @NYKchi88 @nykon187@ofarther"))

['@lindathomas102', '@louisa_ip', '@maga_417', '@MarcGriff89', '@MarjorieLoy1@Maximus_Paulus', '@MC_Mad_Man', '@mgw81180', '@MrSteveRangel', '@MyMountainTop', '@Namashary', '@netteKnows', '@NotUnderdog', '@NYKchi88']


In [13]:
dates = []
user = []
status = []
comments = []
likes = []
retweets = []
tweet = []
tweet_no_html = []
hashtags = []
mentions = []
for tweet_status in tweets_list:
    pos = tweet_status.find(',')
    dates.append(tweet_status[:pos])
    
    tweet_status = tweet_status[pos + 1:].strip()
    pos = tweet_status.find(',')
    user.append(tweet_status[:pos])
    
    tweet_status = tweet_status[pos + 1:].strip()
    pos = tweet_status.find(',')
    status.append(tweet_status[:pos])
    
    tweet_status = tweet_status[pos + 1:].strip()
    pos = tweet_status.find(',')
    comments.append(tweet_status[:pos])
    
    tweet_status = tweet_status[pos + 1:].strip()
    pos = tweet_status.find(',')
    likes.append(tweet_status[:pos])
    
    tweet_status = tweet_status[pos + 1:].strip()
    pos = tweet_status.find(',')
    retweets.append(tweet_status[:pos])
        
    tweet_status = tweet_status[pos + 1:].strip()    
    soup = BeautifulSoup(add_space_between_a_tags(tweet_status),'lxml')
    
    tweet_status_no_html = soup.text
    
    tweet.append(tweet_status_no_html)
    
    pos_2 = tweet_status_no_html.find('pic.twitter.com/')
    tweet_status_no_html = tweet_status_no_html[:pos_2].strip()
    pos_3 = tweet_status_no_html.find('http')
    while pos_3 != -1:        
        tweet_status_no_html = tweet_status_no_html[:pos_3 - 1] + ' ' + tweet_status_no_html[pos_3:]
        pos_3 = tweet_status_no_html.find('http',pos_3 + 1)
    tweet_status_no_html = re.sub(r"http\S+", "", tweet_status_no_html)
    tweet_status_no_html = tweet_status_no_html.strip().replace("\xa0","")
    tweet_no_html.append(tweet_status_no_html)
    
    hashtags.append(get_hashtags(tweet_status_no_html))
    mentions.append(get_mentions(tweet_status_no_html))

In [14]:
dates[1]

'2018-01-01'

# Removing short tweets

In [15]:
count = 0
index = 0
for t in tweet_no_html:
    f = t
    if len(f.lower().replace("donaldtrump","").replace(" ","").replace(".","")) < 150 or int(retweets[index]) < 10 or int(likes[index]) < 20 or int(comments[index]) < 10:
        del(dates[index])
        del(user[index])
        del(status[index])
        del(comments[index])
        del(likes[index])
        del(retweets[index])
        del(tweet[index])
        del(tweet_no_html[index])
        del(hashtags[index])
        del(mentions[index])
        count = count + 1
    index = index + 1
print(count)
print(len(dates))

5040
5204


In [16]:
x = randint(0, len(dates))
print(x)
print(dates[x],user[x],status[x],comments[x],likes[x],retweets[x],tweet[x],tweet_no_html[x],hashtags[x],mentions[x],end="\n")

2711
2018-03-25 trump__toons https://twitter.com/trump__toons/status/977721243980873728 0 2 2 "I'm sorry, sir. We're all out of 'Angry Tangerine'. I've made sacrifices!" -  #donaldtrump pic.twitter.com/eOSitNIPUu  "I'm sorry, sir. We're all out of 'Angry Tangerine'. I've made sacrifices!" -  #donaldtrump [] []


# Grouping by day

In [17]:
dates_by_day = []
user_by_day = []
status_by_day = []
comments_by_day = []
likes_by_day = []
retweets_by_day = []
tweet_by_day = []
tweet_no_html_by_day = []
hashtags_by_day = []
mentions_by_day = []

i = 0
index = 0
current_date = dates[0]
flag = True
for d in dates:
    if (index == 0 and flag) or dates[i] != current_date:
        dates_by_day.append([dates[i]])
        user_by_day.append([user[i]])
        status_by_day.append([status[i]])
        comments_by_day.append([comments[i]])
        likes_by_day.append([likes[i]])
        retweets_by_day.append([retweets[i]])
        tweet_by_day.append([tweet[i]])
        tweet_no_html_by_day.append([tweet_no_html[i]])
        hashtags_by_day.extend([hashtags[i]])
        mentions_by_day.extend([mentions[i]])        
        
        if i != 0:
            index = index + 1
        current_date = dates[i]
        flag = False
    else:
        #print(tweets,index)
        #dates_by_day.append([dates[i]])
        user_by_day[index].append(user[i])
        status_by_day[index].append(status[i])
        comments_by_day[index].append(comments[i])
        likes_by_day[index].append(likes[i])
        retweets_by_day[index].append(retweets[i])
        tweet_by_day[index].append(tweet[i])
        tweet_no_html_by_day[index].append(tweet_no_html[i])
        hashtags_by_day[index].extend(hashtags[i])
        mentions_by_day[index].extend(mentions[i]) 
    i = i + 1
    

In [18]:
tweet_no_html_by_day[0]

['#America  demands  #TrumpShowUsYourPenis #DonaldTrump #MAGA',
 '#DonaldTrump  stop blaming PAK... and PAK Should  #StopAmericanSupplies',
 '#DonaldTrump #DonaldTrumpsPak  Short term memory loss syndrome OR Bipolar disorder?',
 'Dear  #DonaldTrump  you know fairly after March 2018.......you will never be able to threatened State of Pakistan ever again.......So now  #NoMoreTrump',
 "Only problem is .... #DonaldTrump  can't be disappeare",
 'Becoz we are busy in lifting "Mukka" from Mukka chawk..all 33 billion dollars are used to vanish MQM frm Karachiwe need more coz MQM is still there.dont say No mOre! #DonaldTrump',
 'ASSH*LE  #LordAdonis : “ #Brexit  is a  #populist  and  #nationalist  spasm worthy of  #DonaldTrump .” No,  @Andrew_Adonis , Brexit is  #democracy  in action something you clearly do NOT Like!  #GoodRiddance  to  #LordGarbage !  #BBC #SKY #NEWS #UK #EU @UKLabour @Conservatives #Labour #Tory',
 "This is the difference between educated  #PTI  and uneducated  #PMLN . K Asi

In [19]:
index = 0
for i in tweet_no_html:
    print(index,':',i)
    index = index + 1

0 : #America  demands  #TrumpShowUsYourPenis #DonaldTrump #MAGA
1 : #DonaldTrump  stop blaming PAK... and PAK Should  #StopAmericanSupplies
2 : #DonaldTrump #DonaldTrumpsPak  Short term memory loss syndrome OR Bipolar disorder?
3 : Dear  #DonaldTrump  you know fairly after March 2018.......you will never be able to threatened State of Pakistan ever again.......So now  #NoMoreTrump
4 : Only problem is .... #DonaldTrump  can't be disappeare
5 : Becoz we are busy in lifting "Mukka" from Mukka chawk..all 33 billion dollars are used to vanish MQM frm Karachiwe need more coz MQM is still there.dont say No mOre! #DonaldTrump
6 : ASSH*LE  #LordAdonis : “ #Brexit  is a  #populist  and  #nationalist  spasm worthy of  #DonaldTrump .” No,  @Andrew_Adonis , Brexit is  #democracy  in action something you clearly do NOT Like!  #GoodRiddance  to  #LordGarbage !  #BBC #SKY #NEWS #UK #EU @UKLabour @Conservatives #Labour #Tory
7 : This is the difference between educated  #PTI  and uneducated  #PMLN . K A

786 : Could be! ?  I thank the good Lord each morning for  #DonaldTrump ! This man has vision
787 : The fact a jerk mistreats a horrible person does not magically make the horrible person not horrible. It just means that horrible and non-horrible people can both be mistreated by jerks. #MegynKelly #DonaldTrump …
788 : Why  #DonaldTrump  was irrelevant in the government shutdown talks
789 : This guy & John Kelly blew up the immigration deal that caused the  #TrumpShutdown .Miller was called “the tiny dictator” by  @morningmika . So that makes Kelly the big dictator bc  @realDonaldTrump  is totally IRRELEVANT. #DemForce #MorningJoe #MondayMorning #Trump #DonaldTrump
790 : “I think  #DonaldTrump  and his associates are in some serious legal jeopardy”  @RepTedLieu  /  @tedlieu  on The  @MSignorile  Show
791 : #ShepherdSmith  from  #FoxNews  blamed  #DonaldTrump  for the government shutdown:
792 : (WATCH) A liberal student who set fire to a daycare center with 33 children inside to "protest

953 : #StateOfTheDream  counter movement will be attended by  @MarkRuffalo @Alyssa_Milano  and more the day before  #DonaldTrump 's  #StateOfTheUnion
954 : US President  @realDonaldTrump ​ drew boos from the audience at the elite Davos summit Friday when he attacked the press, repeating his charge that critical media indulge in "fake" news.  #DonaldTrump #7News
955 : Man our president is amazing. I pray God's  Protection over him. He's doing so much for the USA! We stand with u  #GreatAwakening #DonaldTrump #WhiteHouse #QAnon
956 : #HappyFriday  FolksYes! He can be my  #President  anytime! #Democrats  choose  #JosephKennedyIII  to respond to  #Trump ’s State of the  #Union  speech  … #FridayFeeling #BarackObama #HillaryClinton #CNN #MSNBC #DonaldTrump #FoxNews   is  #FakeNews ! #FridayFeeling #GOP
957 : #Americans  know that  #DonaldTrump  is the laziest  #POTUS  in US History.  … #FarronCousins #VacationerandChief
958 : #DonaldTrump  is for sale to the highest bidder. We break down ho

1786 : #BREAKING  JUST POSTED on YouTube CHANNEL = jrlcorsi = Feb 22 THURSDAY EVENING Dr.  #JeromeCorsi  DECODE - STARTLING NEW  #EVIDENCE  ON  #SCALA #MURDER …  … VERY  #DISTURBING , VERY  #IMPORTANT  -- Is it True?  #QAnon #Qanon8chan #DonaldTrump @WhiteHouse #bohemiangrove
1787 : . @realDonaldTrump  Is Now Monogramming His Shirts And More With ’45’. Donald Trump is publicizing the fact that he is the 45th president with personalized cuffs, hats, and more.  #DonaldTrump #Interesting
1788 : I want teachers armed w knowledge & government to have exclusive control over  #WeaponsOfWar . This message was not paid for by  #DonaldTrump
1789 : #PCHR  Weekly Report On  #Israeli  Human Rights Violations in the Occupied  #Palestinian  Territory (15 – 21 February 2018)– IMEMC News  … #WestBank #Jerusalem #FreePalestine  #Palestine #Gaza #BoycottIsrael #BDS #CNN #BBC #FoxNews #ABC #NBC #CBS #DonaldTrump
1790 : #DonaldTrump  Boldly Suggests Movies, Video Games Should Get a “Rating System” - Vanity

1952 : Why, I do declare,  #POTUS #DonaldTrump , has it finally come time to say bye-bye to  #JeffSessions ? …
1953 : #DonaldTrump  is meeting with video game makers. $10 says he'll tell them they should have some kind of rating system, because I'm guessing he doesn't know what the  #ESRB  i
1954 : That is Unbelievable! We need our own Website. How do we do it?    #Conservatives #DonaldTrump @RealJamesWoods #OANN #FoxNews
1955 : Donald’s Day Off.Limited Edition Print Release!! Now available on the Lumps store. Every print is signed, numbered & comes with Lumps stickers. The first print order also comes with an original sketch!(Link in bio) #donaldtrump
1956 : Hahaha! I can't believe he said he'd run into a school shooting unarmed!  @realDonaldTrump @TeamCoco @JohnOliverBot @sethmeyers #DonaldTrump #Trump #POTUS
1957 : Seriously  @MSNBC @HallieJackson ?!?  Why do you think that  @mschlapp  is credible to take questions about the  #DonaldTrump  White House?  His wife works there and is t

...

3119 : The only disgrace is  #DonaldTrump . He does not get to decide who gets investigated in this country, and he is not above the law.  #LockHimUp
3120 : #DonaldTrump  insists he is no  #DougFord , and threatens legal action if the comparisons continue. …
3121 : Brace yourself America! What insane thing will  #DonaldTrump  do tonight to try to distract from  #MichaelCohenRaid  and  #StormyDaniels . My bet is on bombing  #Syria  after saying others could deal with it.  His unhinged lunatic is about to show
3122 : BREAKING NEWS! TRUMP IS STILL A JOBBY! And we are still raising money for  @Refuweegee  with £5 from each going straight to them.  … #trump #DonaldTrump #miniprotest
3123 : The guy who told  #DonaldTrump  to stop lying MY HERO!!
3124 : Being  #DonaldTrump 's lawyer is great for business.  People will be breaking down your doors.  #MichaelCohen
3125 : NYT: FBI raids Trump lawyer Michael Cohen's office  #BREAKING #ohshiiiiiit #MuellerIsComing #muellerishere #dontmesswiththefbi

3785 : #BlacChyna  to have her third child with 18 y.o. rapper! Plus, it seems like  #KhloeKardashian  and  #TristanThompson  are over! And all of today's hottest topics! Watch this video in full HERE:    Plus, Lots of  #Kanye  news! Lots of  #DonaldTrump  news! And MORE!
3786 : #RobertMueller  has answers to each of those questions corroborated by evidence & testimony; for drooling, stupid, reflexive liar  #DonaldTrump  to answer any of those questions, he must guess at what a smart prosecutor knows. In technical terms that's called "The Loser's Corner."  …
3787 : Your entire life has been a WITCH HUNT!I can’t imagine you’ve had much time where someone wasn’t coming after you for something. That’s how vile you are.  #Trump  #DonaldTrump  #ResistTrump  #ImpeachTrump  #ImpeachTrumpPENCE  #TrumpIsALiar  #TrumpsWitchHunt
3788 : Trump: Netanyahu’s Press Conference Proves Pulling Out of Iran Deal Maybe Right Choice  #AngelaMerkel #DonaldTrump #Iran #IranDeal #Irannuclearprogram #Israel #Mid

3952 : BREAKING NEWS: The Clinton hypocrisy on illegal immigration  #maga #Trump #donaldtrump #democrats #borderwall #Mexican #conservative #usa #american #PatriotNation #jobsusa #IllegalImmigration #ClintonCrimes #Clintons #HillaryClinton #BillClinton #MaxineWaters
3953 : Donald Trump and Theresa May 'looking forward to UK visit'.  #TheresaMay #DonaldTrump #US #UK #Israel  planning yet another war against  #Muslims  ? #Iran …
3954 : Powerful statement.  I hope Donald Trump respects McCain’s wishes.  But some how I doubt it. Because Donald Trump is a dick.   … #mccain #donaldtrump #TrumpRussiaCollusion #uspoli #cnn #foxnews #cdnmedia
3955 : Grand theft auto in progress... these girls joyriding on a stolen motorbike in London Fields  #nogozonelondon #DonaldTrump . Terrified for my life.
3956 : #TrumpPence , #PresidentTrump , #POTUS , @POTUS , #Persist , #DonaldTrump ,America's political demons hate our heroes, our brothers, and our sisters.
3957 : So did  #DonaldTrump 's team beat his a

4951 : 
4952 : Who will surpass Donald Trump as the most social president – a prediction    by  @IMoyse #facebook #markzuckerberg #donaldtrump #socialmedia
4953 : 
4954 : #DonaldTrump  has granted clemency to Alice Johnson, a first-time nonviolent drug offender who had been given a life sentence without parole.  #kuwtk #ENews
4955 : We now have our version of  #DonaldTrump  =  #DougFord . God help us all.  #OntarioElection2018
4956 : I can't stop laughing...  #US #president #donaldtrump
4957 : “You can’t expect to work for  #DonaldTrump  and not suffer the consequences.” -  #KarounDemirjian , from  @washingtonpost  , on  #CNN .  @karoun @CNN
4958 : Anyone still wondering if  #DonaldTrump  is Russia's puppet? I guess they got their monies worth!!
4959 : A lot of reactions in the  #media  :  #DonaldTrump  will leave the  #G7  before the end, because of tensions with his partners  #G7Summit #International #Canada #G7Charlevoix #Trump #Macron #Trudeau #G6plus1
4960 : The 22 women who havea

5118 : President  #MoonJaeIn  on June11, 1day ahead of the  #SingaporeSummit  , said, “It was the bold determination of the two leaders, President  #DonaldTrump  & Chairman  #KimJongUn , that has enabled us to come this far” #TrumpKoreaSummit #trumpkimsummit See more at: …
5119 : IMPORTANT NEWS BULLET IN - The Message to  #Betfair  is going Worldwide will it be discussed at the  #TrumpKimSummit  We hope  #KimJongUn  will raise it with  #DonaldTrump
5120 : You've made me proud  #RobertDeNiro !  F---  #DonaldTrump !
5121 : We are delighted to announce the winner of  #WeGold  Island Escape: With  #TrumpKimSummit  in  #Singapore who will be visiting from Dubai to celebrate this moment of  #WorldPeace Congratulations! #SingaporeSummit #WeGoGo #cryptocurrency #Travel #Ethereum #DonaldTrump
5122 : BREAKING: Shep Smith To Be Canned Because He Can’t Control His Hate For  #DonaldTrump  - Real Time News...  …
5123 : Can  #DonaldTrump  post something positive, especially to those who are suffering

In [20]:
import nltk
import re
import time
from collections import defaultdict
from configparser import ConfigParser
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from string import digits
import mpld3
import numpy as np
import pandas as pd
nltk.download('punkt')
stemmer = SnowballStemmer('english')
stopword = nltk.corpus.stopwords.words('english')
stopword = stopword + ['trump','donaldtrump','donald','realdonaldtrump']

[nltk_data] Downloading package punkt to /home/miguel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [22]:
totalvocab_stemmed = []
totalvocab_tokenized = []
tweets_day = []
for t in tweet_no_html_by_day:
    i = "".join(t).replace("."," ").replace("\'","")
    tweets_day.append(i)
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [23]:
totalvocab_tokenized

['america',
 'demands',
 'trumpshowusyourpenis',
 'donaldtrump',
 'maga',
 'donaldtrump',
 'stop',
 'blaming',
 'pak',
 'and',
 'pak',
 'should',
 'stopamericansupplies',
 'donaldtrump',
 'donaldtrumpspak',
 'short',
 'term',
 'memory',
 'loss',
 'syndrome',
 'or',
 'bipolar',
 'disorder',
 'dear',
 'donaldtrump',
 'you',
 'know',
 'fairly',
 'after',
 'march',
 'you',
 'will',
 'never',
 'be',
 'able',
 'to',
 'threatened',
 'state',
 'of',
 'pakistan',
 'ever',
 'again',
 'so',
 'now',
 'nomoretrumponly',
 'problem',
 'is',
 'donaldtrump',
 'cant',
 'be',
 'disappearebecoz',
 'we',
 'are',
 'busy',
 'in',
 'lifting',
 'mukka',
 'from',
 'mukka',
 'chawk',
 'all',
 'billion',
 'dollars',
 'are',
 'used',
 'to',
 'vanish',
 'mqm',
 'frm',
 'karachiwe',
 'need',
 'more',
 'coz',
 'mqm',
 'is',
 'still',
 'there',
 'dont',
 'say',
 'no',
 'more',
 'donaldtrumpassh*le',
 'lordadonis',
 'brexit',
 'is',
 'a',
 'populist',
 'and',
 'nationalist',
 'spasm',
 'worthy',
 'of',
 'donaldtrump',


In [24]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

In [25]:
vocab_frame.head

<bound method NDFrame.head of                                     words
america                           america
demand                            demands
trumpshowusyourpeni  trumpshowusyourpenis
donaldtrump                   donaldtrump
maga                                 maga
donaldtrump                   donaldtrump
stop                                 stop
blame                             blaming
pak                                   pak
and                                   and
pak                                   pak
should                             should
stopamericansuppli   stopamericansupplies
donaldtrump                   donaldtrump
donaldtrumpspak           donaldtrumpspak
short                               short
term                                 term
memori                             memory
loss                                 loss
syndrom                          syndrome
or                                     or
bipolar                           bipolar
diso

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8,
                                 min_df=0.2, stop_words=stopword,
                                 use_idf=True, tokenizer=tokenize_and_stem)

%time tfidf_matrix = tfidf_vectorizer.fit_transform(tweets_day) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 2.58 s, sys: 0 ns, total: 2.58 s
Wall time: 2.58 s
(163, 289)


In [27]:
terms = tfidf_vectorizer.get_feature_names()

In [28]:
from sklearn.cluster import KMeans

num_clusters = 30

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 1.79 s, sys: 92.3 ms, total: 1.88 s
Wall time: 523 ms


In [29]:
tweets_info = { 'status': tweets_day, 'hashtags': hashtags_by_day, 'date': dates_by_day, 'mentions': mentions_by_day}

frame = pd.DataFrame(tweets_info, index = [clusters],columns=['status','hashtags','date','mentions','cluster'])

In [30]:
#print(frame['cluster'].value_counts())

In [36]:
from __future__ import print_function
from collections import Counter

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        print('\033[1m',end="")
        print(' %s' % vocab_frame.loc[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
        print('\033[0m',end="")
    print() #add whitespace
    #print() #add whitespace
    
    print("Cluster %d hashtags:" % i, end='')
    h_list_cluster = []
    for h_list in frame.loc[i]['hashtags'].values.tolist():
        h_list_cluster  = h_list_cluster + h_list
    #remove hashtag search criteria
    h_list_cluster = [ t for t in h_list_cluster if t.lower() != "#donaldtrump" and t.lower() != "#trump" and t.lower() != "#potus"]
    print(Counter(h_list_cluster).most_common(10))
    
    print(frame.loc[i]['date'].values.tolist())
    print() #add whitespace
    #print() #add whitespace
    
print()
print()

Top terms per cluster:

Cluster 0 words: b'government', b'america', b'immigrants', b'fbi', b'mueller', b'gop',
Cluster 0 hashtags:[('#TrumpShutdown', 9), ('#MAGA', 8), ('#GOP', 6), ('#CNN', 6), ('#FBI', 6), ('#Democrats', 5), ('#America', 5), ('#ImpeachTrump', 5), ('#TrumpRussia', 5), ('#DACA', 4)]
[['2018-01-20'], ['2018-01-23'], ['2018-01-24'], ['2018-05-27']]

Cluster 1 words: b'want', b'qanon', b'vote', b'state', b'white', b'anyone',
Cluster 1 hashtags:[('#MAGA', 17), ('#USA', 7), ('#QAnon', 7), ('#TheResistance', 5), ('#GOP', 5), ('#EmmanuelMacron', 4), ('#Russia', 4), ('#KimJongUn', 4), ('#BarackObama', 3), ('#ImpeachTrump', 3)]
[['2018-01-08'], ['2018-02-12'], ['2018-02-28'], ['2018-04-25'], ['2018-05-18'], ['2018-05-23']]

Cluster 2 words: b'nra', b'gun', b'russia', b'would', b'house', b'cnn',
Cluster 2 hashtags:[('#NRA', 13), ('#FoxNews', 12), ('#CNN', 10), ('#MAGA', 10), ('#GOP', 9), ('#Palestine', 8), ('#ABC', 8), ('#USA', 8), ('#Russia', 8), ('#FreePalestine', 7)]
[['2018-0

Cluster 25 hashtags:[('#Resist', 14), ('#MAGA', 13), ('#ImpeachTrump', 12), ('#TheResistance', 10), ('#TrumpTrain', 8), ('#Resistance', 7), ('#realDonaldTrump', 6), ('#NotMyPresident', 6), ('#StormyDaniels', 6), ('#AmericaFirst', 5)]
[['2018-01-30'], ['2018-02-13'], ['2018-02-14'], ['2018-02-19'], ['2018-03-08'], ['2018-04-02']]

Cluster 26 words: b'government', b'obama', b'act', b'resist', b'gop', b'said',
Cluster 26 hashtags:[('#TrumpShutdown', 6), ('#WomensMarch2018', 4), ('#Obama', 4), ('#MAGA', 4), ('#GOP', 4), ('#Resist', 3), ('#FakeNews', 3), ('#DACA', 3), ('#TrumpRussia', 3), ('#MelaniaTrump', 3)]
[['2018-01-21'], ['2018-02-03'], ['2018-02-11']]

Cluster 27 words: b'life', b'making', b'america', b'one', b'only', b'trumptrain',
Cluster 27 hashtags:[('#MAGA', 13), ('#FBI', 6), ('#USA', 6), ('#TrumpTrain', 6), ('#TheResistance', 5), ('#maga', 5), ('#militaryparade', 5), ('#FoxNews', 4), ('#Obama', 3), ('#NRA', 3)]
[['2018-01-19'], ['2018-02-07'], ['2018-02-08'], ['2018-02-27'], ['